In [1]:
import pandas as pd
import os
from glob import glob

In [3]:
#Reading in multiple csv files of Citi Bike Trip History Logs (2018 - 2021)
filenames = glob('JC*.csv')
citibikes = pd.concat((pd.read_csv(f) for f in filenames), ignore_index = True)

ValueError: No objects to concatenate

#### Data Cleaning 

In [ ]:
citibikes.head()

In [ ]:
#Ensuring that all column names are meaningful and descriptive
citibikes.columns

In [ ]:
#Checking for nill values
citibikes.isnull().values.any()

In [ ]:
citibikes.dtypes

In [ ]:
#Converting dates and time to pandas datetime type
citibikes['starttime'] = pd.to_datetime(citibikes['starttime'], format='%Y-%m-%d %H:%M:%S.%f')
citibikes['stoptime'] = pd.to_datetime(citibikes['stoptime'], format='%Y-%m-%d %H:%M:%S.%f')
citibikes['birth year'] = pd.to_datetime(citibikes['birth year'], format='%Y')

In [ ]:
#Extract trip year to a different column for timeseries graph later
citibikes['trip year'] = citibikes['starttime'].map(lambda x: x.strftime('%Y'))

In [ ]:
#Confirming starttime, stoptime and birth year are all in pandas datetime type
citibikes.dtypes

In [ ]:
citibikes.head()

In [ ]:
citibikes['trip year'].unique()

In [ ]:
#Replacing the gender column with the citibike key for gender - > (Zero=unknown; 1=male; 2=female)
citibikes['gender']= citibikes['gender'].replace({1 : 'Male', 2: 'Female', 0: 'Unknown' })

In [ ]:
#Checking that all the keys are replaced 
citibikes['gender'].unique()

In [ ]:
#Sorting citibikes dataframe by starttime 
citibikes = citibikes.sort_values(by=['starttime']).reset_index(drop = True)

In [ ]:
citibikes.head()

In [ ]:
# Save citibikes data into a csv file
citibikes.to_csv("citibikes.csv", index = False)